# Issue a Credential using BIP 322 Signatures

This notebook outlines how a Bitcoin address can be used to sign a verifiable credential using BIP 322 signatures explored in other notebooks. Initially, this will just look at using a bitcoin address as the issuer although eventually this should transition to a did:btcr (v2?)

## Notebook Author

* Will Abramson
* [Legendary Requirements](http://legreq.com/)
* Contact: will@legreq.com

## Acknowledgements

This work was funded by Ryan Grant and Digital Contract Design. Thanks also go to Joe Andrieu, Kalle Alm, Pieter Wuille and Jimmy Song for engaging with and supporting various aspects of this work.

In [1]:
cd ..

/home/will/work/LegendaryRequirements/clients/dcd/bip0322-sigs


## Steps

0. Create did:btcr
1. Define VC
2. Canonicalise VC 
3. BIP 322 sign VC payload
4. Attach signature to VC as proof

## 0. Create did:btcr

The issuer needs a DID with a verificationMethod defining the bitcoin scriptPubKey they wish to use as the `message_challenge` in the bip0322 signature algorithm. The issuer needs to be able to "unlock" this scripPubKey. 

For example, if the scriptPubKey is a pay-2-witness-public-key-hash then they need to have the associated private key.

In [2]:
from buidl.ecc import PrivateKey
private_key_wif = "L4qmTC36US6tngrqbrW6vffZbxUVSfipQqXxdJw8S5HqyxNebaMP"
private_key = PrivateKey.parse(private_key_wif)

In [3]:
### TODO: Note using pay-to-witness-public-key-hash address. Should be able to in theory use ANY btc script

In [4]:
address = private_key.point.p2wpkh_address()

In [5]:
# Example test net did:btcr (taken from 0.1 spec)
issuer_did_btcr = "did:btcr:xyv2-xzpq-q9wa-p7t"

In [6]:
# Currently resolves to this
# NOTE: This DID Document currently defines a Ecdsa Secp256k1 public key NOT a bitcoin scriptPubKey
# We need btcr 2.0 to support verificationMethods that are bitcoin scriptPubKeys. Not Secp256k1 public keys
# Question: How do we want to support this?
did_doc = {
    "@context": ["https://w3id.org/did/v0.11", "https://w3id.org/btcr/v1"],
    "id": "did:btcr:xyv2-xzpq-q9wa-p7t",
    "verificationMethod": [
       {
            "id": "did:btcr:xyv2-xzpq-q9wa-p7t#satoshi",
            "controller": "did:btcr:xyv2-xzpq-q9wa-p7t",
            "type": "EcdsaSecp256k1VerificationKey2019",
            "publicKeyBase58": "owh12LKNuphe97teJTZKQTKNewSVTwjHcskPbq34epCY"
        },
        {
             "id": "did:btcr:xyv2-xzpq-q9wa-p7t#vckey-0",
             "controller": "did:btcr:xyv2-xzpq-q9wa-p7t",
             "type": "EcdsaSecp256k1VerificationKey2019",
             "publicKeyBase58": "owh12LKNuphe97teJTZKQTKNewSVTwjHcskPbq34epCY"
         },
    ##########################      EXAMPLE       #################################
        ### What are the resolution rules that parse the Bitcoin Tx at ref xyv2-xzpq-q9wa-p7t
        ### To populate this verificationMethod into the didDocument.
        ### E.g. scriptPubKey from UTX0 at index 1 in tx at ref xyv2-xzpq-q9wa-p7t
        {
             "id": "did:btcr2:xyv2-xzpq-q9wa-p7t#vm-1",
             "controller": "did:btcr:xyv2-xzpq-q9wa-p7t",
             "type": "BIP322",
             "address": address
        }
    
    ###############################################################################
    ],
    "authentication": ["#satoshi"],
    "assertionMethod": ["#vckey-0","#vm-1"]
}


## 1. Define VC


Issuer needs to decide:

* Contents of credential
* Schema of credential
* Key pair and associated identifier they will be signing with
* Identifier to use to the credential subject


In [7]:
#     nVersion = 0
#     nLockTime = 0
#     vin[0].prevout.hash = 0000...000
#     vin[0].prevout.n = 0xFFFFFFFF
#     vin[0].nSequence = 0
#     vin[0].scriptSig = OP_0 PUSH32[ message_hash ]
#     vin[0].scriptWitness = []
#     vout[0].nValue = 0
#     vout[0].scriptPubKey = message_challenge

In [8]:
unsigned_vc = {
  
  "@context": [
    "https://www.w3.org/2018/credentials/v1",
    "https://www.w3.org/2018/credentials/examples/v1"
  ],
  
  "id": "http://example.edu/credentials/1872",
  


  "type": ["VerifiableCredential", "AlumniCredential"],
  

  "issuer": issuer_did_btcr,
    

  "issuanceDate": "2010-01-01T19:23:24Z",
  


  "credentialSubject": {
    

    # TODO: How identify credential subject?
    "id": "did:example:ebfeb1f712ebc6f1c276e12ec21",
    "alumniOf": {
      "id": "did:example:c276e12ec21ebfeb1f712ebc6f1",
      "name": [{
        "value": "Example University",
        "lang": "en"
      }, {
        "value": "Exemple d'Université",
        "lang": "fr"
      }]
    }
  },
  


}

## 2. Canonicalize VC

Not exactly sure what this contains. Need a way to determinstically turn JSONLD VC object into a message to be signed.

**Do I need to implement the URDNA RDF Canonicalization algorithm?** https://json-ld.github.io/rdf-dataset-canonicalization/spec/#canonicalization-algorithm

- https://json-ld.github.io/rdf-dataset-canonicalization/spec/
- https://www.w3.org/TR/vc-data-model/#json-ld

In [14]:
# Naive approach?

# import json
# from buidl.helper import str_to_bytes

# vc_payload = json.dumps(unsigned_vc)
# print(str_to_bytes(vc_payload))

### URDNA Cannonicalization using [PyLD library](https://github.com/digitalbazaar/pyld)

In [15]:
from pyld import jsonld

In [16]:
normalized = jsonld.normalize(
    unsigned_vc, {'algorithm': 'URDNA2015', 'format': 'application/n-quads'})

In [17]:
normalized

'<did:example:c276e12ec21ebfeb1f712ebc6f1> <http://schema.org/name> _:c14n0 .\n<did:example:c276e12ec21ebfeb1f712ebc6f1> <http://schema.org/name> _:c14n1 .\n<did:example:ebfeb1f712ebc6f1c276e12ec21> <http://schema.org/alumniOf> <did:example:c276e12ec21ebfeb1f712ebc6f1> .\n<http://example.edu/credentials/1872> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <https://example.org/examples#AlumniCredential> .\n<http://example.edu/credentials/1872> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <https://www.w3.org/2018/credentials#VerifiableCredential> .\n<http://example.edu/credentials/1872> <https://www.w3.org/2018/credentials#credentialSubject> <did:example:ebfeb1f712ebc6f1c276e12ec21> .\n<http://example.edu/credentials/1872> <https://www.w3.org/2018/credentials#issuanceDate> "2010-01-01T19:23:24Z"^^<http://www.w3.org/2001/XMLSchema#dateTime> .\n<http://example.edu/credentials/1872> <https://www.w3.org/2018/credentials#issuer> <did:btcr:xyv2-xzpq-q9wa-p7t> .\n'

## 3. Sign VC

**Code for sign_message can be found in [message.py](../../edit/src/message.py)**

In [18]:
from src.message import sign_message, MessageSignatureFormat

vc_sig = sign_message(MessageSignatureFormat.SIMPLE, private_key, address, normalized)
print("\n\nBIP 322 Simple Signature on VC", vc_sig)



BIP 322 Simple Signature on VC AkgwRQIhAOnhdQ6NK73jWCSiEXgXOXJxoG2LeZwQ9KZVAbPr0/ALAiAs/Jzx5bp4voCDSUpPPrlpZM8gw/yO7jPq67VURxEXiAEhA8iCJXTXxwVa3nx91ZjdWvR+cjVLVuP6HiwsPPAl9s8x


## 4. Attach Signature to VC as proof

In the BIP322 Signature Suite (TBD) we would define three proof types?

* BIP322LegacySignature2022 
* BIP322SimpleSignature2022
* BIP322FullSignature2022

**Note: Actually perhaps it only requires a BIP322Signature2022 definition. From the signature provided the implementation determines whether it is LEGACY,SIMPLE or FULL**

These types would define how the proof JSON should be structured and interpretted

### 4.1. TODO: what are the acceptable verification methods? How would a verifier be able to resolve to a bitcoin address usable to verify a BIP 322 signature

In [19]:


proof = {

    "type": "BIP322Signature2022",
    


    "created": "2022-05-23T21:19:10Z",
    


    "proofPurpose": "assertionMethod",
    

    
    # This verification method would identify a UTXO on the bitcoin network?
    "verificationMethod": "did:btcr:xyv2-xzpq-q9wa-p7t#vckey-1",
    


    "bip322_sig": vc_sig
}

In [20]:
signed_vc = unsigned_vc
signed_vc["proof"] = proof
print("\n\nCopy this signed VC JSON to the verify notebook \n\n")
print(json.dumps(signed_vc))



Copy this signed VC JSON to the verify notebook 


{"@context": ["https://www.w3.org/2018/credentials/v1", "https://www.w3.org/2018/credentials/examples/v1"], "id": "http://example.edu/credentials/1872", "type": ["VerifiableCredential", "AlumniCredential"], "issuer": "did:btcr:xyv2-xzpq-q9wa-p7t", "issuanceDate": "2010-01-01T19:23:24Z", "credentialSubject": {"id": "did:example:ebfeb1f712ebc6f1c276e12ec21", "alumniOf": {"id": "did:example:c276e12ec21ebfeb1f712ebc6f1", "name": [{"value": "Example University", "lang": "en"}, {"value": "Exemple d'Universit\u00e9", "lang": "fr"}]}}, "proof": {"type": "BIP322Signature2022", "created": "2022-05-23T21:19:10Z", "proofPurpose": "assertionMethod", "verificationMethod": "did:btcr:xyv2-xzpq-q9wa-p7t#vckey-1", "bip322_sig": "AkgwRQIhAOnhdQ6NK73jWCSiEXgXOXJxoG2LeZwQ9KZVAbPr0/ALAiAs/Jzx5bp4voCDSUpPPrlpZM8gw/yO7jPq67VURxEXiAEhA8iCJXTXxwVa3nx91ZjdWvR+cjVLVuP6HiwsPPAl9s8x"}}


## BIP322 Legacy Example : Sign using p2pkh 

### TODO: Signature should encode the public key. (See bitcoin core [SignCompact](https://github.com/bitcoin/bitcoin/blob/48eec32347494da781f26478fa488b28336afbd2/src/key.cpp#L255) function)

In [ ]:
p2pkh_address = private_key.point.address()
p2pkh_address

In [ ]:
vc_sig = sign_message(MessageSignatureFormat.LEGACY, private_key, p2pkh_address, vc_payload)
print("\n\nBIP 322 LEGACY Signature on VC : \n", vc_sig)

In [ ]:
proof = {

    "type": "BIP322Signature2022",
    


    "created": "2022-05-23T21:19:10Z",
    


    "proofPurpose": "assertionMethod",
    

    
    # This verification method would identify a UTXO on the bitcoin network?
    "verificationMethod": "did:btcr:xyv2-xzpq-q9wa-p7t#vckey-1",
    


    "bip322_sig": vc_sig
}

In [ ]:
signed_vc = unsigned_vc
signed_vc["proof"] = proof
print(json.dumps(signed_vc))

## BIP 322 FULL Example

In [ ]:
# Note: if trust SIMPLE with p2pkh_address it is converted to FULL, because no witness to encode
full_vc_sig = sign_message(MessageSignatureFormat.SIMPLE, private_key, p2pkh_address, vc_payload)
print("\n\nBIP 322 FULL Signature on VC : \n", full_vc_sig)

In [ ]:
proof = {

    "type": "BIP322Signature2022",
    


    "created": "2022-05-23T21:19:10Z",
    


    "proofPurpose": "assertionMethod",
    

    
    # This verification method would identify a UTXO on the bitcoin network?
    "verificationMethod": "did:btcr:xyv2-xzpq-q9wa-p7t#vckey-1",
    


    "bip322_sig": full_vc_sig
}

In [ ]:
signed_vc = unsigned_vc
signed_vc["proof"] = proof
print(json.dumps(signed_vc))

In [ ]:
from buidl.cecc import *